# rACS in Real-Multiplexer

In [1]:
# Logger
import logging
logging.basicConfig(level=logging.INFO)

# Import local paths
import sys, os
sys.path.append(os.path.abspath('../../..'))
sys.path.append(os.path.abspath('../../../../openai-envs'))

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS module
from lcs.agents.racs import RACS, Configuration

# Load OpenAI environments
import gym
import gym_multiplexer

## 3-bit Real-Multiplexer
In 3-bit multiplexer first bit is responsible for addressing the correct answer.

> For testing purposes the last bit is used as a flag indicating if the action chosen by the algorithm was correct. The environment always introduces it with 0. It's value is changed to 1 if the action was chosen correctly.

In [2]:
# Environment initialization
rmpx = gym.make('real-multiplexer-3bit-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


rMPX acts like a normal MPX with the difference that instead on boolean values floating numbers within range `[0,1]` are used. Environment used internal threshold to transform them into boolean values (`threshold=0.5` is used as default).

In [3]:
state = rmpx.reset()
state

[0.4863466535210663, 0.7676478988757155, 0.36250925049392413, 0]

Which corresponds to the following binary MPX:

In [4]:
threshold = 0.5  # secret knowledge
binary_state = map(lambda x: 1 if x > threshold else 0, state)

list(binary_state)

[0, 1, 0, 0]

Depending on the MPX size we have some actions to be executed. In 3-bit variant first bit (_address bit_) points to the correct answer (two possible values).

In [5]:
rmpx.action_space.n

2

We can also investigate the properties of a single observation (like vector dimension, lower and upper bound of each attribute).

In [6]:
print(f"Observation shape: {rmpx.observation_space.shape}")
print(f"Upper bounds: {rmpx.observation_space.low}")
print(f"Lower bounds: {rmpx.observation_space.high}")

Observation shape: (4,)
Upper bounds: [0. 0. 0. 0.]
Lower bounds: [1. 1. 1. 1.]


## rACS

- write abount _encoders_

In [7]:
def print_encoding(resolution):
    from lcs.representations.RealValueEncoder import RealValueEncoder
    encoder = RealValueEncoder(resolution_bits=resolution)
    print(f"Encoding values [0,1] using {resolution} bits")
    for i in range (0, 11, 1):
        val = i/10.0
        print(f"Value: {val}\tencoded: {encoder.encode(val)}")
    print("\n")
    
print_encoding(2)
print_encoding(3)

Encoding values [0,1] using 2 bits
Value: 0.0	encoded: 0
Value: 0.1	encoded: 0
Value: 0.2	encoded: 0
Value: 0.3	encoded: 0
Value: 0.4	encoded: 1
Value: 0.5	encoded: 1
Value: 0.6	encoded: 1
Value: 0.7	encoded: 2
Value: 0.8	encoded: 2
Value: 0.9	encoded: 2
Value: 1.0	encoded: 3


Encoding values [0,1] using 3 bits
Value: 0.0	encoded: 0
Value: 0.1	encoded: 0
Value: 0.2	encoded: 1
Value: 0.3	encoded: 2
Value: 0.4	encoded: 2
Value: 0.5	encoded: 3
Value: 0.6	encoded: 4
Value: 0.7	encoded: 4
Value: 0.8	encoded: 5
Value: 0.9	encoded: 6
Value: 1.0	encoded: 7




In the experiment each attribute range will be represented as an _unorded bounded tuple_ with desired encoding resolution.

In [8]:
ENCODER_BITS = 3

cfg = Configuration(rmpx.observation_space.shape[0], rmpx.action_space.n,
                    encoder_bits=ENCODER_BITS,
                    epsilon=0.5,  # exploration-exploitation ratio
                    do_ga=True,
                    theta_r=0.9,  # reliablity threshold
                    theta_i=0.2,  # inadequacy threshold
                    theta_ga=100,
                    chi=0.5,  # cross-over probability
                    mu=0.15)  # mutation probability

Initialize agent with no initial classifiers using configuration above.

In [9]:
agent = RACS(cfg)

Perform exploration for some trials

In [10]:
%%time

EXPLORE_TRIALS = 10_000
population, metrics = agent.explore(rmpx, EXPLORE_TRIALS)

INFO:root:{'agent': {'population': 1, 'numerosity': 1, 'reliable': 0, 'fitness': 0.0, 'cover_ratio': 1.0, 'trial': 0, 'steps': 1, 'total_steps': 1}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 89, 'numerosity': 89, 'reliable': 0, 'fitness': 125.69072909331588, 'cover_ratio': 0.5730337078651685, 'trial': 25, 'steps': 1, 'total_steps': 26}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 294, 'numerosity': 294, 'reliable': 0, 'fitness': 174.35271792096756, 'cover_ratio': 0.5212585034013606, 'trial': 50, 'steps': 1, 'total_steps': 51}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 554, 'numerosity': 554, 'reliable': 0, 'fitness': 174.62623805910883, 'cover_ratio': 0.45216606498194944, 'trial': 75, 'steps': 1, 'total_steps': 76}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 1050, 'numerosity': 1050, 'reliable': 0, 'fitness': 188.08851060843696, 'cover_ratio': 0.39928

INFO:root:{'agent': {'population': 3219, 'numerosity': 3219, 'reliable': 0, 'fitness': 274.9738918615932, 'cover_ratio': 0.1358008254559976, 'trial': 900, 'steps': 1, 'total_steps': 901}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 3194, 'numerosity': 3194, 'reliable': 0, 'fitness': 275.2112191176892, 'cover_ratio': 0.1345826997048033, 'trial': 925, 'steps': 1, 'total_steps': 926}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 3216, 'numerosity': 3216, 'reliable': 0, 'fitness': 273.77941310181126, 'cover_ratio': 0.13451714641080276, 'trial': 950, 'steps': 1, 'total_steps': 951}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 3163, 'numerosity': 3163, 'reliable': 0, 'fitness': 275.5150920468692, 'cover_ratio': 0.13272887403459607, 'trial': 975, 'steps': 1, 'total_steps': 976}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 3176, 'numerosity': 3176, 'reliable': 0, '

INFO:root:{'agent': {'population': 2880, 'numerosity': 2884, 'reliable': 39, 'fitness': 291.776708181534, 'cover_ratio': 0.16076388888888815, 'trial': 1800, 'steps': 1, 'total_steps': 1801}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2878, 'numerosity': 2882, 'reliable': 39, 'fitness': 291.4723806969235, 'cover_ratio': 0.16004417750421848, 'trial': 1825, 'steps': 1, 'total_steps': 1826}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2886, 'numerosity': 2890, 'reliable': 52, 'fitness': 291.7382086786639, 'cover_ratio': 0.16064003564003493, 'trial': 1850, 'steps': 1, 'total_steps': 1851}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2888, 'numerosity': 2892, 'reliable': 53, 'fitness': 292.6921334408231, 'cover_ratio': 0.15953947368420987, 'trial': 1875, 'steps': 1, 'total_steps': 1876}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2867, 'numerosity': 2872, 'rel

INFO:root:{'agent': {'population': 2868, 'numerosity': 2899, 'reliable': 202, 'fitness': 307.03292108972647, 'cover_ratio': 0.1584229926280129, 'trial': 2675, 'steps': 1, 'total_steps': 2676}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2842, 'numerosity': 2877, 'reliable': 209, 'fitness': 308.19890328989055, 'cover_ratio': 0.15678093897657522, 'trial': 2700, 'steps': 1, 'total_steps': 2701}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2837, 'numerosity': 2873, 'reliable': 210, 'fitness': 307.93087024699213, 'cover_ratio': 0.1554333047988311, 'trial': 2725, 'steps': 1, 'total_steps': 2726}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2827, 'numerosity': 2863, 'reliable': 217, 'fitness': 307.8457009210702, 'cover_ratio': 0.1541260296124103, 'trial': 2750, 'steps': 1, 'total_steps': 2751}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2838, 'numerosity': 2875,

INFO:root:{'agent': {'population': 2715, 'numerosity': 2811, 'reliable': 240, 'fitness': 320.1887324569485, 'cover_ratio': 0.14976322020520863, 'trial': 3550, 'steps': 1, 'total_steps': 3551}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2707, 'numerosity': 2806, 'reliable': 241, 'fitness': 321.85607190402396, 'cover_ratio': 0.1486621985329037, 'trial': 3575, 'steps': 1, 'total_steps': 3576}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2684, 'numerosity': 2785, 'reliable': 237, 'fitness': 320.23606479496107, 'cover_ratio': 0.1482994464551838, 'trial': 3600, 'steps': 1, 'total_steps': 3601}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2655, 'numerosity': 2757, 'reliable': 237, 'fitness': 320.8635750822725, 'cover_ratio': 0.14718859295130446, 'trial': 3625, 'steps': 1, 'total_steps': 3626}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2643, 'numerosity': 2747,

...

INFO:root:{'agent': {'population': 2351, 'numerosity': 2527, 'reliable': 207, 'fitness': 300.34936315039647, 'cover_ratio': 0.13246642765996205, 'trial': 7050, 'steps': 1, 'total_steps': 7051}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2340, 'numerosity': 2517, 'reliable': 207, 'fitness': 303.1147829706918, 'cover_ratio': 0.12983821733821704, 'trial': 7075, 'steps': 1, 'total_steps': 7076}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2290, 'numerosity': 2472, 'reliable': 206, 'fitness': 304.9898821893962, 'cover_ratio': 0.12593574547722974, 'trial': 7100, 'steps': 1, 'total_steps': 7101}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2298, 'numerosity': 2481, 'reliable': 205, 'fitness': 303.0508522012644, 'cover_ratio': 0.12863670272286423, 'trial': 7125, 'steps': 1, 'total_steps': 7126}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2293, 'numerosity': 2480

INFO:root:{'agent': {'population': 2119, 'numerosity': 2298, 'reliable': 180, 'fitness': 313.6122160370556, 'cover_ratio': 0.11894087507584444, 'trial': 7925, 'steps': 1, 'total_steps': 7926}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2193, 'numerosity': 2373, 'reliable': 181, 'fitness': 307.78245665085143, 'cover_ratio': 0.1337697869845613, 'trial': 7950, 'steps': 1, 'total_steps': 7951}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2152, 'numerosity': 2331, 'reliable': 179, 'fitness': 308.3958041025613, 'cover_ratio': 0.12909917684545938, 'trial': 7975, 'steps': 1, 'total_steps': 7976}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2142, 'numerosity': 2320, 'reliable': 179, 'fitness': 309.9857978744307, 'cover_ratio': 0.12733426704014938, 'trial': 8000, 'steps': 1, 'total_steps': 8001}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2093, 'numerosity': 2264,

INFO:root:{'agent': {'population': 2006, 'numerosity': 2172, 'reliable': 171, 'fitness': 310.0322448408058, 'cover_ratio': 0.12585457911978362, 'trial': 8800, 'steps': 1, 'total_steps': 8801}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2006, 'numerosity': 2171, 'reliable': 171, 'fitness': 312.2106492175548, 'cover_ratio': 0.12535607463324322, 'trial': 8825, 'steps': 1, 'total_steps': 8826}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2007, 'numerosity': 2172, 'reliable': 171, 'fitness': 313.8672739747168, 'cover_ratio': 0.12493771798704544, 'trial': 8850, 'steps': 1, 'total_steps': 8851}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2002, 'numerosity': 2171, 'reliable': 171, 'fitness': 314.68177174405963, 'cover_ratio': 0.12585628657057235, 'trial': 8875, 'steps': 1, 'total_steps': 8876}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 1970, 'numerosity': 2140

INFO:root:{'agent': {'population': 2009, 'numerosity': 2175, 'reliable': 162, 'fitness': 313.9782712536507, 'cover_ratio': 0.12433335703619443, 'trial': 9675, 'steps': 1, 'total_steps': 9676}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2005, 'numerosity': 2174, 'reliable': 162, 'fitness': 317.2003202425607, 'cover_ratio': 0.12205201282508026, 'trial': 9700, 'steps': 1, 'total_steps': 9701}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2015, 'numerosity': 2182, 'reliable': 162, 'fitness': 315.6698473062448, 'cover_ratio': 0.12286423254165198, 'trial': 9725, 'steps': 1, 'total_steps': 9726}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 1999, 'numerosity': 2167, 'reliable': 161, 'fitness': 317.6168131948543, 'cover_ratio': 0.12147145001071974, 'trial': 9750, 'steps': 1, 'total_steps': 9751}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 2043, 'numerosity': 2211,

CPU times: user 17min 25s, sys: 5.17 s, total: 17min 30s
Wall time: 17min 49s


During the process the algorithm is gathering metrics. Let's inspect the last one

In [11]:
metrics[-1]

{'agent': {'population': 1958,
  'numerosity': 2129,
  'reliable': 168,
  'fitness': 319.59022012624064,
  'cover_ratio': 0.11909017948343786,
  'trial': 9999,
  'steps': 1,
  'total_steps': 10000},
 'environment': None,
 'performance': None}

In [16]:
reliable = [cl for cl in population if cl.is_reliable()]
reliable = sorted(reliable, key=lambda cl: -cl.fitness)

for cl in reliable[:10]:
    print(cl.condition)
    print(cl.effect)
    print(cl.q, cl.r, cl.exp, cl.num)
    
    print("\n")

UBR(x1=4, x2=6)UBR(x1=0, x2=7)UBR(x1=4, x2=5)UBR(x1=0, x2=1)
UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=7, x2=7)
0.9999999999999989 999.9999999999976 537 9


UBR(x1=4, x2=6)UBR(x1=0, x2=7)UBR(x1=5, x2=6)UBR(x1=0, x2=2)
UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=7, x2=7)
0.9999999999997579 999.9999993994824 326 4


UBR(x1=4, x2=6)UBR(x1=0, x2=7)UBR(x1=1, x2=1)UBR(x1=0, x2=0)
UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=7, x2=7)
0.9999999999903991 999.9999842824917 310 6


UBR(x1=0, x2=2)UBR(x1=1, x2=1)UBR(x1=0, x2=7)UBR(x1=0, x2=0)
UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=7, x2=7)
0.9999999997646577 999.9999698780201 240 9


UBR(x1=0, x2=2)UBR(x1=0, x2=0)UBR(x1=0, x2=7)UBR(x1=0, x2=0)
UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=7, x2=7)
0.999999951926071 999.9999851939592 154 9


UBR(x1=1, x2=2)UBR(x1=0, x2=0)UBR(x1=0, x2=7)UBR(x1=0, x2=0)
UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=0, x2=7)UBR(x1=7, x2=7)
0.9999999797021207 999.9998656233162 261 1



Try to exploit

In [13]:
EXPLOIT_TRIALS = 100

# Create new using the whole population
exploiter = RACS(cfg, population)
population_1, metrics_1 = exploiter.exploit(rmpx, EXPLOIT_TRIALS)

INFO:root:{'agent': {'population': 1958, 'numerosity': 2129, 'reliable': 168, 'fitness': 319.60196977711, 'cover_ratio': 0.11909017948343786, 'trial': 0, 'steps': 1, 'total_steps': 1}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 1958, 'numerosity': 2129, 'reliable': 168, 'fitness': 320.48079831859127, 'cover_ratio': 0.11909017948343786, 'trial': 25, 'steps': 1, 'total_steps': 26}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 1958, 'numerosity': 2129, 'reliable': 168, 'fitness': 321.4283604198963, 'cover_ratio': 0.11909017948343786, 'trial': 50, 'steps': 1, 'total_steps': 51}, 'environment': None, 'performance': None}
INFO:root:{'agent': {'population': 1958, 'numerosity': 2129, 'reliable': 168, 'fitness': 321.5917492707432, 'cover_ratio': 0.11909017948343786, 'trial': 75, 'steps': 1, 'total_steps': 76}, 'environment': None, 'performance': None}
